In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/restaurant-revenue-prediction/test.csv.zip
/kaggle/input/restaurant-revenue-prediction/train.csv.zip
/kaggle/input/restaurant-revenue-prediction/sampleSubmission.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df_train = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv.zip')
df_test = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/test.csv.zip')
df_train.shape, df_test.shape

((137, 43), (100000, 42))

In [5]:
df_train

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,132,06/25/2008,Trabzon,Other,FC,2,3.0,3.0,5.0,4,...,3.0,0,0,0,0,0,0,0,0,5787594.0
133,133,10/12/2006,İzmir,Big Cities,FC,4,5.0,4.0,4.0,2,...,3.0,0,0,0,0,0,0,0,0,9262754.0
134,134,07/08/2006,Kayseri,Other,FC,3,4.0,4.0,4.0,2,...,3.0,0,0,0,0,0,0,0,0,2544857.0
135,135,10/29/2010,İstanbul,Big Cities,FC,4,5.0,4.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,7217634.0


In [6]:
train_revenue = df_train.pop('revenue')

### Missing Values

In [7]:
df_train.isnull().sum().sort_values(ascending=False)

P37           0
P6            0
P14           0
P13           0
P12           0
P11           0
P10           0
P9            0
P8            0
P7            0
P5            0
P36           0
P4            0
P3            0
P2            0
P1            0
Type          0
City Group    0
City          0
Open Date     0
P15           0
P16           0
P17           0
P18           0
P35           0
P34           0
P33           0
P32           0
P31           0
P30           0
P29           0
P28           0
P27           0
P26           0
P25           0
P24           0
P23           0
P22           0
P21           0
P20           0
P19           0
Id            0
dtype: int64

In [8]:
df_test.isnull().sum().sort_values(ascending=False)

P37           0
P6            0
P14           0
P13           0
P12           0
P11           0
P10           0
P9            0
P8            0
P7            0
P5            0
P36           0
P4            0
P3            0
P2            0
P1            0
Type          0
City Group    0
City          0
Open Date     0
P15           0
P16           0
P17           0
P18           0
P35           0
P34           0
P33           0
P32           0
P31           0
P30           0
P29           0
P28           0
P27           0
P26           0
P25           0
P24           0
P23           0
P22           0
P21           0
P20           0
P19           0
Id            0
dtype: int64

#### No missing values in either train or test

### Working on Date Column

- The Idea is instead of having whole date, lets only have year as the value

In [9]:
df_train['Open Date'] = df_train['Open Date'].str.split('/').apply(lambda x : x[2])

In [10]:
df_test['Open Date'] = df_test['Open Date'].str.split('/').apply(lambda x : x[2])

In [11]:
df_train.shape, df_test.shape

((137, 42), (100000, 42))

In [12]:
df_train.drop(columns=["Id"],inplace=True)
df_test_index = df_test.pop('Id')

In [13]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')

In [14]:
df_train_ohe = ohe.fit_transform(df_train)
df_train_ohe = df_train_ohe.todense()

In [15]:
df_test_ohe = ohe.transform(df_test)
df_test_ohe = df_test_ohe.toarray()

In [16]:
df_train_ohe.shape

(137, 345)

In [17]:
df_test_ohe.shape

(100000, 345)

### Model Building

In [18]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

In [19]:
param = {
    "n_estimators": range(10,50),
    "learning_rate": [0.0001,0.001,0.01,0.1,1],
    "loss" : ['linear', 'square', 'exponential']
}

In [20]:
abr = AdaBoostRegressor(random_state=17)

In [21]:
#abr_gcv = GridSearchCV(abr,param,'neg_mean_squared_error',cv=5)
#abr_gcv.fit(df_train_ohe,train_revenue)

In [22]:
#abr_gcv.best_estimator_

In [23]:
abr = AdaBoostRegressor(base_estimator=None, learning_rate=0.00001, loss='exponential',
                  n_estimators=14, random_state=17)

In [24]:
abr.fit(df_train_ohe,train_revenue)

AdaBoostRegressor(base_estimator=None, learning_rate=1e-05, loss='exponential',
                  n_estimators=14, random_state=17)

In [25]:
test_revenue = abr.predict(df_test_ohe)

In [26]:
df_submit = pd.DataFrame({'Id': df_test_index, 'Prediction': test_revenue})

In [27]:
df_submit.to_csv('submit.csv',index=False) 
df_submit.head()

,Id,Prediction
0,0,3.930053e+06
1,1,3.482235e+06
2,2,3.581191e+06
3,3,3.736490e+06
4,4,3.736490e+06
